In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/welfake-dataset/WELFake_Dataset.csv


In [3]:
df = pd.read_csv('/kaggle/input/welfake-dataset/WELFake_Dataset.csv')
df.head()


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
df.drop("Unnamed: 0", axis = 1)
df.dropna(inplace = True)

In [6]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [4]:
#removing other than alphates and digits
import re
df['title'] = df['title'].str.replace(r"[^\w\s]", "").str.replace(r"\d", "")
df['text'] = df['text'].str.replace(r"[^\w\s]","").str.replace(r"\d", "")
df.head()

/tmp/ipykernel_32/3271493751.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r"[^\w\s]", "").str.replace(r"\d", "")
/tmp/ipykernel_32/3271493751.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r"[^\w\s]","").str.replace(r"\d", "")


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE OBAMAS ATTORNEY GENERAL SAYS MOST...,Now most of the demonstrators gathered last n...,1
3,3,Bobby Jindal raised Hindu uses story of Christ...,A dozen politically active pastors came here f...,0
4,4,SATAN Russia unvelis an image of its terrifyi...,The RS Sarmat missile dubbed Satan will repla...,1
5,5,About Time Christian Group Sues Amazon and SPL...,All we can say on this one is it s about time ...,1


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


nltk.download('stopwords')
nltk.download('punkt')

# Function to tokenize and remove stopwords from text
def tokenize_and_remove_stopwords(text):
    # Convert the text to lowercase to ensure consistency
    text = text.lower()

    # Get the set of English stopwords from NLTK
    stop_words = set(stopwords.words('english'))

    # Tokenize the text into words
    tokens = word_tokenize(text)

    # Remove stopwords from the list of tokens
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Reconstruct the text by joining the remaining tokens
    filtered_text = " ".join(filtered_tokens)

    return filtered_text

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df['title'] = df['title'].apply(tokenize_and_remove_stopwords)
df['text'] = df['text'].apply(tokenize_and_remove_stopwords)

In [5]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


# using glove

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6b/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

texts = df['text'].values + df['title'].values
labels = df['label'].values

max_words = 10000  # Vocabulary size
max_len = 100  # Maximum sequence length

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

word_index = tokenizer.word_index

num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, 100))  # Assuming you're using 100-dimensional GloVe embeddings

for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Create GloVe embedding sequences for each text sample
glove_sequences = np.array([np.mean([embedding_matrix[word_index[word]] for word in sequence], axis=0)
                            for sequence in sequences])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(glove_sequences, labels, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


KeyError: 423

In [13]:
X = df['title'] + df['text']
y = df['label']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state = 0,  stratify = y) 

In [16]:
max_words = 10000 #total number of words in the dictionary
max_len = 1000 # maximum length of each seq after padding

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

tokenizer= Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train) 

# Step 2: Get the total number of unique words in the vocabulary
total_words = len(tokenizer.word_index)

# Step 3: Get the occurrences of each word
word_counts = tokenizer.word_counts

# Step 4: Print the results
print("Total number of words in the vocabulary:", total_words)
'''print("Occurrences of each word:")
for word, count in word_counts.items():
    print(f"{word}: {count}")'''

Total number of words in the vocabulary: 261662


'print("Occurrences of each word:")\nfor word, count in word_counts.items():\n    print(f"{word}: {count}")'

In [17]:

training_sequences = tokenizer.texts_to_sequences(X_train)  # converting text to sequence of integers
training_padded = sequence.pad_sequences(training_sequences,maxlen=max_len)  

test_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [18]:
# creating the dictionary having words as keys and corr vector as values of words which are in glove
embeddings_index = dict()
f = open('/kaggle/input/glove6b/glove.6B.100d.txt', encoding = "utf8")
for line in f:    
    values = line.split() #breaking each line into word and number (each line has first word then numbers representing that vector)
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
print (len(embeddings_index))
f.close()

400000


In [19]:
embedding_matrix = np.zeros((max_words, 100))
for word, index in tokenizer.word_index.items():
    if index > max_words - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
print (embedding_matrix)


[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.026284    0.017938   -0.50164998 ... -0.86879998  0.94094002
  -0.28815001]
 [-0.042568   -0.084246   -0.33094999 ... -0.92315     0.012559
   0.21473999]
 ...
 [ 0.36258999  0.0089486   0.56475002 ... -0.082847   -0.41046
   0.56173003]
 [-0.58547002 -0.4932     -0.20903    ... -0.49373999 -0.97579998
  -0.51665998]
 [-0.50588     0.89183998  0.0041605  ... -0.58831    -0.22042
   0.083857  ]]


In [20]:
embedding_features = embedding_matrix.reshape(embedding_matrix.shape[0], -1)

In [21]:
print(embedding_features)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.026284    0.017938   -0.50164998 ... -0.86879998  0.94094002
  -0.28815001]
 [-0.042568   -0.084246   -0.33094999 ... -0.92315     0.012559
   0.21473999]
 ...
 [ 0.36258999  0.0089486   0.56475002 ... -0.082847   -0.41046
   0.56173003]
 [-0.58547002 -0.4932     -0.20903    ... -0.49373999 -0.97579998
  -0.51665998]
 [-0.50588     0.89183998  0.0041605  ... -0.58831    -0.22042
   0.083857  ]]


In [22]:
print(testing_padded)

[[   0    0    0 ...  121   55  121]
 [1889   26 2702 ...  955  620  645]
 [   0    0    0 ... 2357 3202 4624]
 ...
 [   0    0    0 ... 1634   78 3133]
 [   0    0    0 ... 3332  114 2297]
 [   0    0    0 ...  340 1227    8]]


# using tf-idf

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 1000)
tf_idf_matrix_h = tfidf.fit_transform(df['title'])
tf_idf_matrix_text = tfidf.fit_transform(df['text'])
title = tf_idf_matrix_h.toarray()
text = tf_idf_matrix_text.toarray()

In [16]:
a = []
for i in title:
    a.append(i)
print(a[1])
df['tf_title'] = a

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.39446798 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [17]:
b = []
for i in text:
    b.append(i)
print(b[1])
df['tf_text'] = b

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [31]:
df['label'].shape

(71537,)

In [18]:
import numpy as np
title = np.vstack(df['tf_title'].values)
text = np.vstack(df['tf_text'].values)
X = np.concatenate((title,text),axis = 1)
y = df['label']
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42, stratify = y)

In [36]:
print(y.shape)

(71537,)


In [37]:
y_train.shape

(57229,)

In [19]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(solver = 'lbfgs', max_iter = 1000)
model_lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [20]:
y_pred = model_lr.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9609309477215544
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      6891
           1       0.97      0.95      0.96      7417

    accuracy                           0.96     14308
   macro avg       0.96      0.96      0.96     14308
weighted avg       0.96      0.96      0.96     14308



In [49]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors = 2, metric = 'minkowski', p=2)
model_knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=2)

In [50]:
y_pred = model_knn.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.7713167458764327
              precision    recall  f1-score   support

           0       0.64      0.86      0.73      5194
           1       0.90      0.72      0.80      9114

    accuracy                           0.77     14308
   macro avg       0.77      0.79      0.77     14308
weighted avg       0.80      0.77      0.78     14308



In [23]:
from sklearn.svm import SVC 
model_svm = SVC(kernel = 'linear', random_state = 0)
model_svm.fit(X_train,y_train)

SVC(kernel='linear', random_state=0)

In [24]:
y_pred = model_svm.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9612804025719877
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      6888
           1       0.97      0.95      0.96      7420

    accuracy                           0.96     14308
   macro avg       0.96      0.96      0.96     14308
weighted avg       0.96      0.96      0.96     14308



In [25]:
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)
y_pred = model_nb.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.8368744758177243
              precision    recall  f1-score   support

           0       0.86      0.81      0.84      7442
           1       0.81      0.86      0.84      6866

    accuracy                           0.84     14308
   macro avg       0.84      0.84      0.84     14308
weighted avg       0.84      0.84      0.84     14308



In [33]:
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
model_dt.fit(X_train, y_train)
y_pred = model_dt.predict(X_test)


In [34]:
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9442270058708415
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      6946
           1       0.95      0.94      0.95      7362

    accuracy                           0.94     14308
   macro avg       0.94      0.94      0.94     14308
weighted avg       0.94      0.94      0.94     14308



# Ensembling algorithms


In [42]:

from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=50, random_state=42)
#model_rf = RandomForsetClassifier(n_estimators = 100, random_state = 42)
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9612804025719877
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      6776
           1       0.98      0.95      0.96      7532

    accuracy                           0.96     14308
   macro avg       0.96      0.96      0.96     14308
weighted avg       0.96      0.96      0.96     14308



In [46]:
import xgboost as xgb
model_xgb = xgb.XGBClassifier(objective = 'binary:logistic',
                             n_estimators = 50,
                             max_depth =3 ,
                             learning_rate = 0.1,
                             random_state = 42)
model_xgb.fit(X_train, y_train)
y_pred = model_xgb.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.9390550740844283
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      6412
           1       0.98      0.91      0.94      7896

    accuracy                           0.94     14308
   macro avg       0.94      0.94      0.94     14308
weighted avg       0.94      0.94      0.94     14308



In [55]:
from sklearn.ensemble import AdaBoostClassifier

base_estimator = DecisionTreeClassifier(max_depth = 3)
model_ad = AdaBoostClassifier(base_estimator =base_estimator, n_estimators = 50, random_state = 42 )
model_ad.fit(X_train, y_train)
y_pred = model_ad.predict(X_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.9582051998881744
              precision    recall  f1-score   support

           0       0.95      0.96      0.96      6932
           1       0.96      0.95      0.96      7376

    accuracy                           0.96     14308
   macro avg       0.96      0.96      0.96     14308
weighted avg       0.96      0.96      0.96     14308

